## csr_matrix ?
- 모델링 직전에

In [1177]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import myslack
from sklearn.ensemble import GradientBoostingRegressor 

In [1178]:
train = pd.read_csv("./data/train.csv")

In [1179]:
test = pd.read_csv("./data/test.csv")

In [1180]:
samplesub = pd.read_csv("./data/sample_submission.csv")

In [1181]:
samplesub.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preprocessing

## Check Train 'NaN' data before Imputation

In [1182]:
train_desc = train.describe(include='all').transpose()
train_desc['MV_count'] = train.isnull().sum()
train_desc['MV_ratio'] = train_desc['MV_count']/train.shape[0]
train_desc[['MV_count','MV_ratio']]

,MV_count,MV_ratio
TripType,0,0.000000
VisitNumber,0,0.000000
Weekday,0,0.000000
Upc,4129,0.006381
ScanCount,0,0.000000
DepartmentDescription,1361,0.002103
FinelineNumber,4129,0.006381


## Check Test 'NaN' data before Imputation

In [1183]:
test_desc = test.describe(include='all').transpose()
test_desc['MV_count'] = test.isnull().sum()
test_desc['MV_ratio'] = test_desc['MV_count']/test.shape[0]
test_desc[['MV_count','MV_ratio']]

,MV_count,MV_ratio
VisitNumber,0,0.000000
Weekday,0,0.000000
Upc,3986,0.006098
ScanCount,0,0.000000
DepartmentDescription,1328,0.002032
FinelineNumber,3986,0.006098


## Preprocessing Step1 - Imputation 
아래 순서처럼 대분류, 중분류, 소분류 순으로 Imputation 한다.
- DepartmentDescription 
- FinelineNumber
- Upc

### 1-1. Train - DepartmentDescription

- VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움

In [1184]:
# 일단 DepartmentDescription에 'MENSWEAR' 와 'MENS WEAR'가 구분되어 있던 것을 'MENS WEAR'로 통일
train.loc[train['DepartmentDescription']=='MENSWEAR','DepartmentDescription']='MENS WEAR'
train[train['DepartmentDescription']=='MENSWEAR']

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [1185]:
# before imputation
train['DepartmentDescription'].isna().sum()

1361

In [1186]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = train[train['DepartmentDescription'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(train[train["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index) != 0:
        train.loc[(train["VisitNumber"] == idx)&(train["DepartmentDescription"].isna()), "DepartmentDescription"] \
        = train[train["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index[0]

100%|██████████| 1172/1172 [00:37<00:00, 31.00it/s]


In [1187]:
# after imputation step1
train['DepartmentDescription'].isna().sum()

191

In [1188]:
# 아직 처리하지 못한 총 191개 데이터는 VisitNumber 따른 DepartmentDescription 값이 모두 비어있는 경우인데
# 이 경우 TripType이 모두 999이 때문에 그런 DepartmentDescription는 TripType 999의 DepartmentDescription 최빈값으로 imputation 함.
train.loc[train["DepartmentDescription"].isna(), "DepartmentDescription"] \
= train[train["TripType"] == 999]["DepartmentDescription"].value_counts().index[0]

In [1189]:
train[train["TripType"] == 999]["DepartmentDescription"].value_counts().index[0]

'FINANCIAL SERVICES'

In [1190]:
# after imputation step2
train['DepartmentDescription'].isna().sum()

0

### 1-2. Test - DepartmentDescription

- VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움

In [1191]:
test = pd.read_csv("./data/test.csv")

In [1192]:
# 일단 DepartmentDescription에 'MENSWEAR' 와 'MENS WEAR'가 구분되어 있던 것을 'MENS WEAR'로 통일
test.loc[test['DepartmentDescription']=='MENSWEAR','DepartmentDescription']='MENS WEAR'
test[test['DepartmentDescription']=='MENSWEAR']

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [1193]:
# before imputation
test['DepartmentDescription'].isna().sum()

1328

In [1194]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = test[test['DepartmentDescription'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(test[test["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index) != 0:
        test.loc[(test["VisitNumber"] == idx)&(test["DepartmentDescription"].isna()), "DepartmentDescription"] \
        = test[test["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index[0]

100%|██████████| 1141/1141 [00:35<00:00, 31.97it/s]


In [1195]:
# after imputation step1
test['DepartmentDescription'].isna().sum()

219

In [1196]:
# 아직 처리하지 못한 총 219개 데이터는 VisitNumber 따른 DepartmentDescription 값이 모두 비어있는 경우인데
# 이 경우 Train데이터에서 TripType이 모두 999이었고 그런 DepartmentDescription의 최빈값인 'FINANCIAL SERVICES'로 imputaion 함.
test.loc[test["DepartmentDescription"].isna(), "DepartmentDescription"] = 'FINANCIAL SERVICES'

In [1197]:
train[train["TripType"] == 999]["DepartmentDescription"].value_counts().index[0]

'FINANCIAL SERVICES'

In [1198]:
# after imputation step2
test['DepartmentDescription'].isna().sum()

0

### 2-1. Train - FinelineNumber

- VisitNumber에 따른 FinelineNumber의 최빈값으로 FinelineNumber의 NaN 값을 채움

In [1199]:
# before imputation
train['FinelineNumber'].isna().sum()

4129

In [1200]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = train[train['FinelineNumber'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(train[train["VisitNumber"] == idx]["FinelineNumber"].value_counts().index) != 0:
        train.loc[(train["VisitNumber"] == idx)&(train["FinelineNumber"].isna()), "FinelineNumber"] \
        = train[train["VisitNumber"] == idx]["FinelineNumber"].value_counts().index[0]

100%|██████████| 2754/2754 [00:15<00:00, 178.30it/s]


In [1201]:
# after imputation step1
train['FinelineNumber'].isna().sum()

2369

In [1202]:
# 아직 처리하지 못한 총 2369개 데이터는 TripType으로 구분해보면 5와 999 두 개로 나뉜다.
train[train['FinelineNumber'].isna()]['TripType'].value_counts()

5      2089
999     280
Name: TripType, dtype: int64

In [1203]:
train_after_step1_FL_null_VM_list = train[train['FinelineNumber'].isna()]['VisitNumber'].value_counts().index
train_after_step1_FL_null_VM_list

Int64Index([ 74003,  52567, 111813, 114942, 156445,  83995,  41497,  79927,
              1860,  69775,
            ...
            168331,  78201, 127351,  78197, 168307,  16747,  14684, 102743,
              2390,  16384],
           dtype='int64', length=1427)

In [1204]:
# 따라서 각 TripType의 최빈 FinelineNumber 값으로 imputation 함.
train.loc[(train["TripType"] == 5)&(train["FinelineNumber"].isna()), "FinelineNumber"] \
= train[train["TripType"] == 5]["FinelineNumber"].value_counts().index[0]
train.loc[(train["TripType"] == 999)&(train["FinelineNumber"].isna()), "FinelineNumber"] \
= train[train["TripType"] == 999]["FinelineNumber"].value_counts().index[0]

In [1205]:
train[train["TripType"] == 5]["FinelineNumber"].value_counts().index[0]

5501.0

In [1206]:
train[train["TripType"] == 999]["FinelineNumber"].value_counts().index[0]

279.0

In [1207]:
# after imputation step2
train['FinelineNumber'].isna().sum()

0

### 2-2. Test - FinelineNumber

- VisitNumber에 따른 FinelineNumber의 최빈값으로 FinelineNumber의 NaN 값을 채움

In [1208]:
# before imputation
test['FinelineNumber'].isna().sum()

3986

In [1209]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = test[test['FinelineNumber'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(test[test["VisitNumber"] == idx]["FinelineNumber"].value_counts().index) != 0:
        test.loc[(test["VisitNumber"] == idx)&(test["FinelineNumber"].isna()), "FinelineNumber"] \
        = test[test["VisitNumber"] == idx]["FinelineNumber"].value_counts().index[0]

100%|██████████| 2706/2706 [00:15<00:00, 176.47it/s]


In [1210]:
# after imputation step1
test['FinelineNumber'].isna().sum()

2257

In [1211]:
# 아직 처리하지 못한 총 2257개 데이터는 VisitNumber 따른 FinelineNumber 값이 모두 비어있는 경우인데
# 이 경우 Train데이터에서 TripType이 5 혹은 999이어서 그에 맞춰 FinelineNumber의 최빈값을 imputation할 수 있었지만
# Test데이터는 TripType을 기준으로 imputation 할 수 없고 train데이터와 VisitNumber와 도 겹치지 않기 때문에
# Train데이터와 Test데이터의 Weekday, ScanCount, DepartmentDescription을 기준으로 FinelineNumber를 imputation 함.

In [1212]:
# train데이터와 test데이터의 imputation step1 이후의 VisitNumber가 겹치지 않음
test_after_step1_FL_null_VM_list = test[test['FinelineNumber'].isna()]['VisitNumber'].value_counts().index
[i for i in test_after_step1_FL_null_VM_list if i in train_after_step1_FL_null_VM_list]

[]

In [1213]:
def test_FinelineNumber_or_Upc_imputation_step2(feature):
    print('Let\'s start '+str(feature)+' imputation\n')
    for weekday in train['Weekday'].unique():
        print('============ Start Weekday: '+str(weekday) +' ============\n')
        for scancount in [2,1,-1,-2,-3,-4,-5]:
            print('------------ Start ScanCount: '+str(scancount)+' ------------')
            for dd in ['PHARMACY RX','FINANCIAL SERVICES']:
                try:
                    print('\n[ DepartmentDescription: \''+str(dd)+'\' ]'
                          +'\nthis combination contains: '
                          +str(len(test.loc[(test[feature].isna())&(test['Weekday']==weekday)
                                       &(test['ScanCount']==scancount)
                                       &(test['DepartmentDescription']==dd),feature])))
                    test.loc[(test[feature].isna())
                         &(test['Weekday']==weekday)
                         &(test['ScanCount']==scancount)
                         &(test['DepartmentDescription']==dd),feature]\
                    = train.loc[(train['Weekday']==weekday)
                            &(train['ScanCount']==scancount)
                            &(train['DepartmentDescription']==dd)
                            &(train['TripType'] == train.loc[(train['Weekday']==weekday)
                                                             &(train['ScanCount']==scancount)
                                                             &(train['DepartmentDescription']==dd),
                                                             'TripType'].value_counts().index[0]),
                                feature].value_counts().index[0]
                    print('after this combination imputation, NaN still remains: '
                          +str(test[feature].isna().sum()))


                except IndexError:
                    print('\n[ DepartmentDescription: \''+str(dd)+'\' ]'
                          +'\nthis combination doesn\'t exist, so let\'s except train scancount condition, then it contains: '
                          +str(len(test.loc[(test[feature].isna())&(test['Weekday']==weekday)
                                       &(test['ScanCount']==scancount)
                                       &(test['DepartmentDescription']==dd),feature])))
                    test.loc[(test[feature].isna())
                         &(test['Weekday']==weekday)
                         &(test['ScanCount']==scancount)
                         &(test['DepartmentDescription']==dd),feature]\
                    = train.loc[(train['Weekday']==weekday)
                            &(train['DepartmentDescription']==dd)
                            &(train['TripType'] == train.loc[(train['Weekday']==weekday)
                                                             &(train['DepartmentDescription']==dd),
                                                             'TripType'].value_counts().index[0]),
                                feature].value_counts().index[0]
                    print('after this combination imputation, NaN still remains: '
                          +str(test[feature].isna().sum()))
            print('\n------------ Finish ScanCount: '+str(scancount)+' ------------\n')
        print('============ Finish Weekday: '+str(weekday)+' ============\n')
    print(str(feature)+' Imputation Finished!')    
    return print('After '+str(feature)+' Imputation, NaN count: '+str(test[feature].isna().sum()))

In [1214]:
test_FinelineNumber_or_Upc_imputation_step2('FinelineNumber')

Let's start FinelineNumber imputation

============ Start Weekday: Friday ============

------------ Start ScanCount: 2 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0
after this combination imputation, NaN still remains: 2257

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 1
after this combination imputation, NaN still remains: 2256

------------ Finish ScanCount: 2 ------------

------------ Start ScanCount: 1 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 376
after this combination imputation, NaN still remains: 1880

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 5
after this combination imputation, NaN still remains: 1875

------------ Finish ScanCount: 1 ------------

------------ Start ScanCount: -1 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 7
after this combination imputation, NaN still remains: 1868

[ DepartmentD


[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 1325

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0
after this combination imputation, NaN still remains: 1325

------------ Finish ScanCount: -2 ------------

------------ Start ScanCount: -3 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0

[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 1325

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0
after this combination imputation, NaN still remains: 1325

------------ Finish ScanCount: -3 ------------

------------ Start ScanCount: -4 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0

[


[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 712

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 712

------------ Finish ScanCount: -4 ------------

------------ Start ScanCount: -5 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0

[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 712

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination doesn't exist, so le

In [1215]:
test[test['FinelineNumber'].isna()]

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [1216]:
# after imputation step2
test['FinelineNumber'].isna().sum()

0

### 3-1. Train - Upc

- VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움

In [1217]:
# before imputation
train['Upc'].isna().sum()

4129

In [1218]:
# Upc이 NaN 값인 데이터의 VisitNumber 리스트
Upc_null_VM = train[train['Upc'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움
for idx in tqdm(Upc_null_VM):
    if len(train[train["VisitNumber"] == idx]["Upc"].value_counts().index) != 0:
        train.loc[(train["VisitNumber"] == idx)&(train["Upc"].isna()), "Upc"] \
        = train[train["VisitNumber"] == idx]["Upc"].value_counts().index[0]

100%|██████████| 2754/2754 [00:15<00:00, 179.71it/s]


In [1219]:
# after imputation step1
train['Upc'].isna().sum()

2369

In [1220]:
# 아직 처리하지 못한 총 2369개 데이터는 TripType으로 구분해보면 5와 999 두 개로 나뉜다.
train[train['Upc'].isna()]['TripType'].value_counts()

5      2089
999     280
Name: TripType, dtype: int64

In [1221]:
train_after_step1_Upc_null_VM_list = train[train['Upc'].isna()]['VisitNumber'].value_counts().index
train_after_step1_Upc_null_VM_list

Int64Index([ 74003,  52567, 111813, 114942, 156445,  83995,  41497,  79927,
              1860,  69775,
            ...
            168331,  78201, 127351,  78197, 168307,  16747,  14684, 102743,
              2390,  16384],
           dtype='int64', length=1427)

In [1222]:
# 따라서 각 TripType의 최빈 Upc 값으로 imputation 함.
train.loc[(train["TripType"] == 5)&(train["Upc"].isna()), "Upc"] \
= train[train["TripType"] == 5]["Upc"].value_counts().index[0]
train.loc[(train["TripType"] == 999)&(train["Upc"].isna()), "Upc"] \
= train[train["TripType"] == 999]["Upc"].value_counts().index[0]

In [1223]:
train[train["TripType"] == 5]["Upc"].value_counts().index[0]

4011.0

In [1224]:
train[train["TripType"] == 999]["Upc"].value_counts().index[0]

60538812238.0

In [1225]:
# after imputation step2
train['Upc'].isna().sum()

0

### 3-2. Test - Upc

- VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움

In [1226]:
# before imputation
test['Upc'].isna().sum()

3986

In [1227]:
# Upc이 NaN 값인 데이터의 VisitNumber 리스트
Upc_null_VM = test[test['Upc'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움
for idx in tqdm(Upc_null_VM):
    if len(test[test["VisitNumber"] == idx]["Upc"].value_counts().index) != 0:
        test.loc[(test["VisitNumber"] == idx)&(test["Upc"].isna()), "Upc"] \
        = test[test["VisitNumber"] == idx]["Upc"].value_counts().index[0]

100%|██████████| 2706/2706 [00:15<00:00, 176.72it/s]


In [1228]:
# after imputation step1
test['Upc'].isna().sum()

2257

In [1229]:
# 아직 처리하지 못한 총 2257개 데이터는 VisitNumber 따른 Upc 값이 모두 비어있는 경우인데
# 이 경우 Train데이터에서 TripType이 5 혹은 999이어서 그에 맞춰 Upc 최빈값을 imputation할 수 있었지만
# Test데이터는 TripType을 기준으로 imputation 할 수 없고 train데이터와 VisitNumber와 도 겹치지 않기 때문에
# Train데이터와 Test데이터의 Weekday, ScanCount, DepartmentDescription을 기준으로 Upc를 imputation 함.

In [1230]:
# train데이터와 test데이터의 imputation step1 이후의 VisitNumber가 겹치지 않음
test_after_step1_Upc_null_VM_list = test[test['Upc'].isna()]['VisitNumber'].value_counts().index
[i for i in test_after_step1_Upc_null_VM_list if i in train_after_step1_Upc_null_VM_list]

[]

In [1231]:
test_FinelineNumber_or_Upc_imputation_step2('Upc')

Let's start Upc imputation

============ Start Weekday: Friday ============

------------ Start ScanCount: 2 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0
after this combination imputation, NaN still remains: 2257

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 1
after this combination imputation, NaN still remains: 2256

------------ Finish ScanCount: 2 ------------

------------ Start ScanCount: 1 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 376
after this combination imputation, NaN still remains: 1880

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 5
after this combination imputation, NaN still remains: 1875

------------ Finish ScanCount: 1 ------------

------------ Start ScanCount: -1 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 7
after this combination imputation, NaN still remains: 1868

[ DepartmentDescription:


[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 1325

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0
after this combination imputation, NaN still remains: 1325

------------ Finish ScanCount: -2 ------------

------------ Start ScanCount: -3 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0

[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 1325

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0
after this combination imputation, NaN still remains: 1325

------------ Finish ScanCount: -3 ------------

------------ Start ScanCount: -4 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0

[


[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 712

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 712

------------ Finish ScanCount: -4 ------------

------------ Start ScanCount: -5 ------------

[ DepartmentDescription: 'PHARMACY RX' ]
this combination contains: 0

[ DepartmentDescription: 'PHARMACY RX' ]
this combination doesn't exist, so let's except train scancount condition, then it contains: 0
after this combination imputation, NaN still remains: 712

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination contains: 0

[ DepartmentDescription: 'FINANCIAL SERVICES' ]
this combination doesn't exist, so le

In [1232]:
test[test['Upc'].isna()]

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [1233]:
# after imputation step2
test['Upc'].isna().sum()

0

## Check Train 'NaN' data after Imputation

In [1234]:
desc = train.describe(include='all').transpose()
desc['MV_count'] = train.isnull().sum()
desc['MV_ratio'] = desc['MV_count']/train.shape[0]
desc[['MV_count','MV_ratio']]

,MV_count,MV_ratio
TripType,0,0.0
VisitNumber,0,0.0
Weekday,0,0.0
Upc,0,0.0
ScanCount,0,0.0
DepartmentDescription,0,0.0
FinelineNumber,0,0.0


## Check Test 'NaN' data after Imputation

In [1235]:
desc = train.describe(include='all').transpose()
desc['MV_count'] = train.isnull().sum()
desc['MV_ratio'] = desc['MV_count']/train.shape[0]
desc[['MV_count','MV_ratio']]

,MV_count,MV_ratio
TripType,0,0.0
VisitNumber,0,0.0
Weekday,0,0.0
Upc,0,0.0
ScanCount,0,0.0
DepartmentDescription,0,0.0
FinelineNumber,0,0.0


In [1236]:
myslack.send_slack('Imputation Finish!!!')

<Response [200]>


## Preprocessing Step2 -  Encoding

### Weekday

In [1237]:
# before
train['Weekday'].value_counts()

Sunday       133975
Saturday     122096
Friday        96247
Monday        83130
Tuesday       72529
Wednesday     71115
Thursday      67962
Name: Weekday, dtype: int64

In [1238]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

train['Weekday'] = weekday_preprocessing(train)
train['Weekday'].value_counts()

1    133975
2    122096
3     96247
4     83130
5     72529
6     71115
7     67962
Name: Weekday, dtype: int64

### ScanCount

- VisitNumber로 groupby 할때 sum으로 대체

In [19]:
# after preprocessing
desc['MV_count'] = train.isnull().sum()
desc['MV_ratio'] = desc['MV_count']/train.shape[0]
desc

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,MV_count,MV_ratio
TripType,647054,NaN,NaN,NaN,58.5845,157.636,3,27,39,40,999,0,0.000000
VisitNumber,647054,NaN,NaN,NaN,96167.6,55545.5,5,49268,97074,144316,191347,0,0.000000
Weekday,647054,NaN,NaN,NaN,3.4997,2.00343,1,2,3,5,7,0,0.000000
Upc,642925,NaN,NaN,NaN,3.0607e+10,9.12013e+10,834,3.4e+09,7.0501e+09,3.00653e+10,9.78971e+11,4129,0.006381
ScanCount,647054,NaN,NaN,NaN,1.10888,0.700776,-12,1,1,1,71,0,0.000000
DepartmentDescription,645693,68,GROCERY DRY GOODS,70402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1361,0.002103
FinelineNumber,642925,NaN,NaN,NaN,3726.88,2780.97,0,1404,3352,5501,9998,4129,0.006381


##  Feature importance via Gradient Boosting model

In [52]:
X = train.drop(columns='TripType')
y = train['TripType']

In [54]:
start_time=time.time()

gb = GradientBoostingRegressor()
gb.fit(X, y)
features = X.columns.values

end_time=time.time()
print("total time in the current cell ",end_time-start_time,"s")

ValueError: could not convert string to float: 'GROCERY DRY GOODS'

In [46]:
# Scatter plot 
trace = go.Scatter(
    y = gb.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        color = gb.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Machine Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)